# Dataset Analysis

In [1]:
import pandas as pd
import numpy as np
from datasets import *

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
dogs = load_skl_dogs_2016(scaler=StandardScaler())
dogs.data.shape

(117, 15)

In [3]:
print("Linear SVR")
svr_lin = svm.SVR(C=4, epsilon=0.1, kernel='linear')
X, y = dogs.data, dogs.target
svr_lin.fit(X,y)
print("Score: %f" % svr_lin.score(X,y))

Linear SVR
Score: -0.046704


In [4]:
print("Rbf SVR")
svr = svm.SVR(C=4, epsilon=0.1, gamma=0.001, kernel='rbf')
X, y = dogs.data, dogs.target
svr.fit(X,y)
print("Score: %f" % svr.score(X,y))

Rbf SVR
Score: -0.124022
